lyft uber yellow year month day hour pulocationid weekday isholiday

In [9]:
import pandas as pd
# 讀取檔案資訊
datasite = '../TS6-201902_202306.parquet'
df = pd.read_parquet(datasite)

print(df.shape)
df.head()

(30273912, 13)


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.84651,2019-02-01 00:00:00,4.0
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.84651,2019-02-01 01:00:00,2.0
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.84651,2019-02-01 02:00:00,6.0
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.84651,2019-02-01 03:00:00,1.0
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.84651,2019-02-01 04:00:00,1.0


In [10]:
#清出基本格式
df_temp = df
df_temp = df_temp.drop(['Name','count','lat','lon','countN'],axis=1)
df_temp.drop_duplicates(inplace=True)
df_temp = df_temp.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
print(df_temp.shape)
df_temp

(10091304, 8)


,year,month,day,hour,PULocationID,weekday,is_holiday,TempTime
0,2019,2,1,0,1,4,False,2019-02-01 00:00:00
1,2019,2,1,1,1,4,False,2019-02-01 01:00:00
2,2019,2,1,2,1,4,False,2019-02-01 02:00:00
3,2019,2,1,3,1,4,False,2019-02-01 03:00:00
4,2019,2,1,4,1,4,False,2019-02-01 04:00:00
...,...,...,...,...,...,...,...,...
10091299,2023,6,30,19,263,4,False,2023-06-30 19:00:00
10091300,2023,6,30,20,263,4,False,2023-06-30 20:00:00
10091301,2023,6,30,21,263,4,False,2023-06-30 21:00:00
10091302,2023,6,30,22,263,4,False,2023-06-30 22:00:00


In [3]:
#lyft
df_lyft = df[df['Name'] == 'lyft']
df_lyft = df_lyft.drop(['Name','year','month','day','hour','weekday','is_holiday','lat','lon','countN'],axis=1)
df_lyft = df_lyft.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_lyft = df_lyft.drop(['PULocationID','TempTime'],axis=1)
df_lyft.rename(columns={'count': 'lyft'}, inplace=True)
#uber
df_uber = df[df['Name'] == 'uber']
df_uber = df_uber.drop(['Name','year','month','day','hour','weekday','is_holiday','lat','lon','countN'],axis=1)
df_uber = df_uber.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_uber = df_uber.drop(['PULocationID','TempTime'],axis=1)
df_uber.rename(columns={'count': 'uber'}, inplace=True)
#yellow
df_yellow = df[df['Name'] == 'yellow']
df_yellow = df_yellow.drop(['Name','year','month','day','hour','weekday','is_holiday','lat','lon','countN'],axis=1)
df_yellow = df_yellow.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_yellow = df_yellow.drop(['PULocationID','TempTime'],axis=1)
df_yellow.rename(columns={'count': 'yellow'}, inplace=True)

In [4]:
df_temp = pd.concat([df_lyft,df_uber,df_yellow, df_temp], axis=1)
df_temp = df_temp.drop(['TempTime'],axis=1)

In [5]:
df_temp.head()

,lyft,uber,yellow,year,month,day,hour,PULocationID,weekday,is_holiday
0,0,0,0,2019,2,1,0,1,4,False
1,0,0,0,2019,2,1,1,1,4,False
2,0,0,0,2019,2,1,2,1,4,False
3,0,0,0,2019,2,1,3,1,4,False
4,0,0,0,2019,2,1,4,1,4,False


In [6]:
#放入地區
datasite_temp = '../taxi_zone_lat_lon/organized_taxi_zone_lat_lon.csv'
#創立一個 df_temp 讀經緯度資料
df_temp1 = pd.read_csv(datasite_temp)

df_temp1

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,40.710736,-73.962940
259,Woodlawn/Wakefield,259,Bronx,40.900107,-73.853635
260,Woodside,260,Queens,40.746439,-73.905907
261,World Trade Center,261,Manhattan,40.707456,-74.013983


In [7]:
df_temp = df_temp.merge(df_temp1[['LocationID', 'zone','borough']], left_on=['PULocationID'], right_on=['LocationID'], how='left')
df_temp = df_temp.drop(['LocationID'],axis = 1)
df_temp['datetime'] = pd.to_datetime(df_temp[['year', 'month', 'day', 'hour']])
df_temp['total'] = df_temp['lyft'] + df_temp['uber'] + df_temp['yellow']
df_temp

,lyft,uber,yellow,year,month,day,hour,PULocationID,weekday,is_holiday,zone,borough,datetime,total
0,0,0,0,2019,2,1,0,1,4,False,Newark Airport,EWR,2019-02-01 00:00:00,0
1,0,0,0,2019,2,1,1,1,4,False,Newark Airport,EWR,2019-02-01 01:00:00,0
2,0,0,0,2019,2,1,2,1,4,False,Newark Airport,EWR,2019-02-01 02:00:00,0
3,0,0,0,2019,2,1,3,1,4,False,Newark Airport,EWR,2019-02-01 03:00:00,0
4,0,0,0,2019,2,1,4,1,4,False,Newark Airport,EWR,2019-02-01 04:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10091299,78,169,82,2023,6,30,19,263,4,False,Yorkville West,Manhattan,2023-06-30 19:00:00,329
10091300,66,113,82,2023,6,30,20,263,4,False,Yorkville West,Manhattan,2023-06-30 20:00:00,261
10091301,52,124,90,2023,6,30,21,263,4,False,Yorkville West,Manhattan,2023-06-30 21:00:00,266
10091302,52,128,89,2023,6,30,22,263,4,False,Yorkville West,Manhattan,2023-06-30 22:00:00,269


In [8]:
df = df_temp
import os
# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'.'
DataName = 'TS20-201902_202306'
# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')

save!


---

In [9]:
df[(df['year']==2019 ) & (df['month']== 2 )& (df['day']== 10 )& (df['PULocationID']== 10 )& (df['Name']== 'lyft' )]

,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN
3576,lyft,2019,2,10,0,10,6,False,19,40.677931,-73.790317,2019-02-10 00:00:00,17.000000
3577,lyft,2019,2,10,1,10,6,False,13,40.677931,-73.790317,2019-02-10 01:00:00,13.333333
3578,lyft,2019,2,10,2,10,6,False,11,40.677931,-73.790317,2019-02-10 02:00:00,8.000000
3579,lyft,2019,2,10,3,10,6,False,10,40.677931,-73.790317,2019-02-10 03:00:00,5.666667
3580,lyft,2019,2,10,4,10,6,False,11,40.677931,-73.790317,2019-02-10 04:00:00,10.000000
3581,lyft,2019,2,10,5,10,6,False,12,40.677931,-73.790317,2019-02-10 05:00:00,11.333333
3582,lyft,2019,2,10,6,10,6,False,11,40.677931,-73.790317,2019-02-10 06:00:00,11.333333
3583,lyft,2019,2,10,7,10,6,False,15,40.677931,-73.790317,2019-02-10 07:00:00,15.333333
3584,lyft,2019,2,10,8,10,6,False,13,40.677931,-73.790317,2019-02-10 08:00:00,11.000000
3585,lyft,2019,2,10,9,10,6,False,12,40.677931,-73.790317,2019-02-10 09:00:00,12.666667


In [8]:
df_temp[(df_temp['year']==2019 ) & (df_temp['month']== 2 )& (df_temp['day']== 10 )& (df_temp['PULocationID']== 10 )]

,lyft,uber,yellow,year,month,day,hour,PULocationID,weekday,is_holiday
348192,19,44,0,2019,2,10,0,10,6,False
348193,13,32,1,2019,2,10,1,10,6,False
348194,11,17,0,2019,2,10,2,10,6,False
348195,10,26,0,2019,2,10,3,10,6,False
348196,11,17,0,2019,2,10,4,10,6,False
348197,12,22,0,2019,2,10,5,10,6,False
348198,11,30,4,2019,2,10,6,10,6,False
348199,15,51,1,2019,2,10,7,10,6,False
348200,13,43,5,2019,2,10,8,10,6,False
348201,12,49,6,2019,2,10,9,10,6,False
